# Evaluation
Both the raw predictions and the ground truth are lists of true and false values.
Here, we compare the predictions and ground truth to derive confusion matrices and scoring metrics.
## Preparation
### Imports

In [1]:
from src import data
from typing import Tuple
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, fbeta_score, accuracy_score, roc_auc_score
from imblearn.metrics import specificity_score

### Loading The Data
Begin by loading the data.
The predictions by the supervised machine learning models and the llama models are within different files.

In [2]:
path_data = '../../../data/datasets/04_preprocessed'
path_predictions = '../../../data/predictions'

datasets = data.dict_from_directory(path_data, type='polars')

# predictions by the supervised machine learning models
preds_sml = data.dict_from_directory(path_predictions + '/supervised_machine_learning', type='polars')

# predictions by the llama model
preds_llama = data.dict_from_directory(path_predictions + '/llama_3_1_8B', type='polars')

The llama predictions initially do not contain the true labels.
Therefore concatenate the true lables and the index number from the dataset.

Further, rename the llama prediction to better distinguish it from the sml predictions:

In [3]:
preds_llama = {
    subject: predictions.rename(
        {
            'include': 'llama',
            'reason': 'llama_reason'
        }
    ).with_columns(
        datasets[subject].select(
            pl.col('index').alias('index'),
            pl.col('include').alias('true_llama')
        )
    )
    for subject, predictions in preds_llama.items()
}

The supervised machine learning models only predicted the classes of articles within the test dataset.

Concatenate the predictions of all models by the indices of the articles within the test set:

In [4]:
joint_test_set = {
    subject: predictions.join(
        preds_llama[subject], on='index'
    ).drop(['true_llama'])
    for subject, predictions in preds_sml.items()
}

The ```joint_test_set``` variable now contains the articles within the sets sets.
For each article there is the ground truth ``true``, the prediction by each respective model as well as llama's reasoning:

In [5]:
joint_test_set['oral_hypoglycemics'].sample(10)

index,true,logistic_regression,random_forest,support_vector_machine,naive_bayes,llama,llama_reason,title,doi,pubmed_id
i64,bool,bool,bool,bool,bool,bool,str,str,str,i64
473,false,true,true,true,true,true,""" The study meets all the inclu…","""The Effect of Magnesium Supple…","""https://doi.org/10.2337/diacar…",9589224
238,false,false,false,false,true,true,""" The study meets all the inclu…","""A Comparison of the Effects of…","""https://doi.org/10.2337/diacar…",12401757
84,false,false,false,false,true,true,""" The study meets all the inclu…","""Less nocturnal hypoglycemia an…","""https://doi.org/10.2337/diacar…",10937510
26,true,true,true,true,true,true,""" The study meets all the inclu…","""A comparison of preconstituted…","""https://doi.org/10.1007/s00592…",10436254
390,false,false,true,false,true,true,""" The study meets all the inclu…","""Circulating catecholamines and…","""https://doi.org/10.2337/diacar…",8742566
99,false,false,false,false,true,true,""" The study meets the inclusion…","""The effect of oral hypoglycaem…","""nan""",11070748
2,false,true,true,true,true,true,""" The study meets all the inclu…","""Shortterm intensive insulin th…","""nan""",10085698
215,false,false,false,false,false,false,""" The article does not contain …","""Desensitization of insulin sec…","""https://doi.org/10.1016/s0006-…",12093468
283,false,false,false,false,false,false,""" The abstract does not mention…","""NN Novo Nordisk""","""nan""",12789615


### Helper Functions

In [6]:
def confusion_matrices(y_true, y_pred) -> Tuple[np.ndarray, np.ndarray]:

    matrix = confusion_matrix(
           y_true=y_true, 
           y_pred=y_pred,
           normalize=None
        )
       
    matrix_norm = confusion_matrix(
        y_true=y_true, 
        y_pred=y_pred,
        normalize='true'
    )

    return matrix, matrix_norm

In [7]:
from sklearn.utils import resample

def generate_bootstrap_samples(
        y_true, # true labels
        y_pred, # predicted labels
        n_resamples=1000, # number of bootstrap samples, 1000 is common
    ):

    # save the bootstrapped samples here
    samples = []

    # create n_resamples bootstrap samples
    for _ in range(n_resamples):
        indices = np.random.choice(len(y_true), len(y_true), replace=True)
        y_true_resampled = y_true[indices]
        y_pred_resampled = y_pred[indices]

        #samples.append((y_true_resampled, y_pred_resampled))
        samples.append(
            {
                'y_true': y_true_resampled,
                'y_pred': y_pred_resampled
            }
        )

    return samples

# TODO: Remove Macro averaging later

In [8]:
def metric_with_ci(
        bootstraps, # list of bootstrapped samples
        metric, # tuple: name and function
        round_ndigits=2, # number of digits to round the results
        confidence_level=0.95 # confidence level for the CI
):
    
    metric_name, metric_function = metric[0], metric[1]

    # save the scores for each bootstrap sample
    scores = []

    # calculate the score for each bootstrap sample
    for index, sample in enumerate(bootstraps):

        y_true = sample['y_true']
        y_pred = sample['y_pred']

        try:
            # calculate the score
            if metric_name == 'f2': # pass beta argument for f2-score
                score = metric_function(y_true, y_pred, beta=2)
            else:
                score = metric_function(y_true, y_pred)
            scores.append(score)
        except:
            score = None
        
    # add the score to the distribution
    scores = [score for score in scores if score is not None]
    scores = np.array(scores) # transform to numpy for calculations
    
    # calculate the mean and lower and upper bounds
    mean = np.mean(scores)
    lower = np.percentile(scores, (1 - confidence_level) / 2 * 100)
    upper = np.percentile(scores, (1 + confidence_level) / 2 * 100)

    # round the results
    mean = round(mean, round_ndigits)
    lower = round(lower, round_ndigits)
    upper = round(upper, round_ndigits)

    return mean, lower, upper, scores

# Evaluate

## Define Structures
### Estimators

In [9]:
estimators = [
    'logistic_regression',
    'random_forest',
    'support_vector_machine',
    'naive_bayes',
    'llama'
]

### Metric Functions

In [10]:
metrics = {
    'accuracy': accuracy_score,
    'precision': precision_score,
    'recall': recall_score,
    'f1': f1_score,
    'f2': fbeta_score,
    'specificity': specificity_score,
}

### Scores Table
The structure for the large table that will save scores and confidence intervals across all datasets and estimators:

In [11]:
schema_context = {
    'dataset': pl.String,
    'estimator': pl.String,
}

schema_metrics = {
    metric: pl.Struct(
        [
            pl.Field(f'{metric}_mean', pl.Float64),
            pl.Field(f'{metric}_lower', pl.Float64),
            pl.Field(f'{metric}_upper', pl.Float64),
        ]
    )
    for metric in metrics.keys()
}

schema_scores = schema_context | schema_metrics

### Results Dictionary
Every piece of the results will be saved within this dictionary for easy access:

In [12]:
results = {
    'scores': pl.DataFrame([], schema=schema_scores),
    'datasets': {
        subject: {
            estimator: {
                'classification_report': None,
                'matrix': {},
                'bootstrap': {
                    'samples': None,
                    'scores': {
                        metric: None
                        for metric in metrics
                    }

                }
            }
            for estimator in estimators
        }
        for subject in joint_test_set.keys()
        
    }
}

In [13]:
results_llama = {
    'scores': pl.DataFrame([], schema=schema_scores),
}

for subject in preds_llama.keys():
    results_llama[subject] = {
        'classification_report': None,
        'matrix': {},
        'bootstrap': {
            'samples': None,
            'scores': {
                metric: None
                for metric in metrics
            }
        }
    }

results_llama.keys()

dict_keys(['scores', 'adhd', 'animal_depression', 'atypical_antipsychotics', 'calcium_channel_blockers', 'oral_hypoglycemics', 'pancreatic_surgery'])

# Calculate Results (TODO: Rework for both testset & LLM)
Calculate the result metrics and save all metrics and accompaning data, such as classification results, confusion matrices, and bootstrap samples, within one results-dictionary:

## Llama on whole datasets

In [14]:
for subject, dataset in tqdm(
    iterable=preds_llama.items(),
    desc='Datasets',
    total=len(preds_llama),
    leave=True,
):
    model = 'llama'

    y_true = dataset['true_llama'].to_numpy()
    estimator = dataset['llama'].to_numpy()

    # classification report
    report = classification_report(y_true, estimator)
    results_llama[subject]['classification_report'] = report


    matrix, matrix_norm = confusion_matrices(y_true, estimator)
    results_llama[subject]['matrix']['absolute'] = matrix
    results_llama[subject]['matrix']['norm'] = matrix_norm

    estimator_scores = {}

    bootstraps = generate_bootstrap_samples(y_true, estimator)
    results_llama[subject]['bootstrap']['samples'] = bootstraps

    for metric, function in tqdm(
        iterable=metrics.items(),
        desc='Metrics',
        total=len(metrics),
        leave=False
    ):

        mean, lower, upper, bootstrap_scores = metric_with_ci(bootstraps, (metric, function))
        results_llama[subject]['bootstrap']['scores'][metric] = bootstrap_scores

        struct_series = pl.Series(
                name=metric,
                values=[(mean, lower, upper)],
                dtype=pl.Struct(
                    [
                        pl.Field(f'{metric}_mean', pl.Float64),
                        pl.Field(f'{metric}_lower', pl.Float64),
                        pl.Field(f'{metric}_upper', pl.Float64),
                    ]
                )
            )

        estimator_scores[metric] = struct_series[0]
    
    scores = pl.DataFrame(
        {
            est: [vals]
            for est, vals in estimator_scores.items()
        }
    )

    scores = scores.with_columns(
        pl.lit(subject, pl.String).alias('dataset'),
        pl.lit(model, pl.String).alias('estimator')
    )

    column_order = ['dataset', 'estimator'] + list(metrics.keys())
    scores = scores.select(column_order)

    results_llama['scores'] = results_llama['scores'].vstack(scores)

Datasets:   0%|          | 0/6 [00:00<?, ?it/s]

Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

In [15]:
results_llama['scores']

dataset,estimator,accuracy,precision,recall,f1,f2,specificity
str,str,struct[3],struct[3],struct[3],struct[3],struct[3],struct[3]
"""adhd""","""llama""","{0.82,0.8,0.85}","{0.12,0.07,0.17}","{1.0,1.0,1.0}","{0.21,0.13,0.3}","{0.4,0.27,0.51}","{0.82,0.79,0.85}"
"""animal_depression""","""llama""","{0.85,0.83,0.87}","{0.49,0.45,0.54}","{0.96,0.93,0.98}","{0.65,0.61,0.69}","{0.8,0.77,0.83}","{0.83,0.81,0.85}"
"""atypical_antipsychotics""","""llama""","{0.51,0.48,0.54}","{0.18,0.15,0.22}","{0.79,0.72,0.85}","{0.3,0.25,0.34}","{0.48,0.42,0.53}","{0.47,0.43,0.5}"
"""calcium_channel_blockers""","""llama""","{0.42,0.39,0.45}","{0.11,0.08,0.13}","{0.81,0.72,0.88}","{0.19,0.15,0.22}","{0.35,0.29,0.4}","{0.38,0.36,0.41}"
"""oral_hypoglycemics""","""llama""","{0.45,0.4,0.49}","{0.31,0.26,0.36}","{0.89,0.84,0.94}","{0.46,0.4,0.52}","{0.65,0.59,0.7}","{0.29,0.24,0.34}"
"""pancreatic_surgery""","""llama""","{0.62,0.61,0.62}","{0.11,0.1,0.11}","{0.8,0.78,0.82}","{0.19,0.18,0.2}","{0.35,0.34,0.36}","{0.61,0.6,0.61}"


## All models on test-sets

In [16]:
# iterate over the datasets
for subject, dataset in tqdm(
    iterable=joint_test_set.items(),  # iterate over the datasets
    desc='Datasets',
    total=len(joint_test_set),
    leave=True
):


    # save the predictions within a variable
    predictions_only = dataset.select(
        pl.exclude(
            ['index', 'true', 'llama_reason', 'title', 'doi', 'pubmed_id']
        )
    )


    # iterate over the estimators
    for estimator in tqdm(
        iterable=predictions_only.iter_columns(),
        desc='Estimators',
        total=len(predictions_only.columns),
        leave=False
    ):
        
        
        
        model = estimator.name  # to access the results dictionary
        
        y_true = dataset['true'].to_numpy() # true labels

        # predictions by the current estimator to calculate on
        estimator = estimator.to_numpy()  

        # add the classification report
        report = classification_report(y_true, estimator)
        results['datasets'][subject][model]['classification_report'] = report

        # add the confusion matrices
        matrix, matrix_norm = confusion_matrices(y_true, estimator)
        results['datasets'][subject][model]['matrix']['absolute'] = matrix
        results['datasets'][subject][model]['matrix']['norm'] = matrix_norm

        # save the scores for each estimator in here
        estimator_scores = {}

        # calculate all metrics on the same set of bootstrap samples
        bootstraps = generate_bootstrap_samples(y_true, estimator)
        
        # add the bootstraps to the results for traceability
        results['datasets'][subject][model]['bootstrap']['samples'] = bootstraps

        # calculate each metric with confidence intervals by bootstrapping
        for metric, function in metrics.items():

            # calculate the metric with confidence intervals
            mean, lower, upper, bootstrap_scores = metric_with_ci(bootstraps, (metric, function))
            results['datasets'][subject][model]['bootstrap']['scores'][metric] = bootstrap_scores

            # Create a Series with the struct values
            struct_series = pl.Series(
                name=metric,
                values=[(mean, lower, upper)],
                dtype=pl.Struct(
                    [
                        pl.Field(f'{metric}_mean', pl.Float64),
                        pl.Field(f'{metric}_lower', pl.Float64),
                        pl.Field(f'{metric}_upper', pl.Float64),
                    ]
                )
            )

            # add the results struct to the list of scores
            estimator_scores[metric] = struct_series[0]

        # add the results to the scores dataframe
        scores = pl.DataFrame({
            estimator: [values]
            for estimator, values in estimator_scores.items()
        })
        
        scores = scores.with_columns(
            pl.lit(subject, pl.String).alias('dataset'),
            pl.lit(model, pl.String).alias('estimator')
        )

        # reorder the dataframe for stacking
        column_order = ['dataset', 'estimator'] + list(metrics.keys())
        scores = scores.select(column_order)

        results['scores'] = results['scores'].vstack(scores)

Datasets:   0%|          | 0/6 [00:00<?, ?it/s]

Estimators:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\root\miniforge3\envs\automated_screening\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\root\miniforge3\envs\automated_screening\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\root\miniforge3\envs\automated_screening\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\root

Estimators:   0%|          | 0/5 [00:00<?, ?it/s]

Estimators:   0%|          | 0/5 [00:00<?, ?it/s]

Estimators:   0%|          | 0/5 [00:00<?, ?it/s]

Estimators:   0%|          | 0/5 [00:00<?, ?it/s]

Estimators:   0%|          | 0/5 [00:00<?, ?it/s]

# Export

In [17]:
with open('./results', 'wb') as file:
    import pickle
    pickle.dump(results, file, protocol=pickle.HIGHEST_PROTOCOL)